# Data Processing and Feature Engineering

In this notebook, we merge the different datasets together, select features and engineer some new features. This first step in feature selection and engineering is based on my knowledge of the data. 

We proceed in this order:  

1. Filter water systems of interest (active water systems in New England)
2. Select features of interest for water systems, and data cleaning
3. Filter violations of interest (pesticides)
4. Select features of interest for violations, and data cleaning
5. Add estimated pesticide use for the water systems
6. Merge water systems and violations to obtain violations by water systems (!)
7. 
8. Engineer new features of interest for violations by water systems 


In the end, we obtain a dateset on which we can do model training and selection. In a later step, when trying different models, a new loop of feature selection and engineering might be needed, and will be performed in a separate notebook.     


In [2]:
import pandas as pd
import numpy as np
import datetime
import time

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import geopandas as gpd # to identify neighboring water systems via ZIP code
from shapely.geometry import Point, Polygon

## Water Systems

We did already select the water systems of interest: EPA region 01, New England, and only active water systems.

In [6]:
# load water systems:
ws_raw = pd.read_csv('../data/active_water_systems_NewEngland.csv')
ws_raw.head().T

,0,1,2,3,4
Unnamed: 0,1,2,3,4,5
pwsid,ME0004628,ME0092288,ME0009198,ME0094505,ME0007311
pws_name,MACHIAS TRAILER PARK,MARSH BROOK ESTATES,TRAILS END STEAK HOUSE & TAVERN,R & R VACATION HOME PARK,NOKOMIS CAMPING AREA LLC
npm_candidate,Y,Y,Y,Y,Y
primacy_agency_code,ME,ME,ME,ME,ME
epa_region,1,1,1,1,1
season_begin_date,NaN,NaN,01-01,01-01,05-01
season_end_date,NaN,NaN,12-31,12-31,09-30
pws_activity_code,A,A,A,A,A
pws_deactivation_date,NaN,NaN,NaN,NaN,NaN


In [93]:
# DATA CLEANING FOR WATER SYSTEMS
ws = ws_raw.copy()

# IMPORTANT NOTE: wer cannot use the ZIP code to locate the water systems, 
# as it is sometimes the ZIP code of the legal entity, which is not necessarily in the same place.

# ws.state_code.value_counts() # cannot be used, as state of legal entity.

# Thus we will locate the water systems, with the counties_served + primacy_agency_code
# ws.primacy_agency_code.value_counts()

# The 5 water systems administred by "tribes" have no city/county served. 
# ws[ws.primacy_agency_code == '01'].T

# We manually impute those, based on the city_name (legal entity):
# ws[ws.counties_served == 'Not Reported']

ws.loc[ws.pwsid == 'VT0021432', 'counties_served'] = 'windham'

ws.loc[ws['pwsid'] == '010307001', 'counties_served'] = 'dukes'
ws.loc[ws['pwsid'] == '010307001', 'cities_served'] = 'chilmark'

ws.loc[ws['pwsid'] == '010106001', 'counties_served'] = 'new london'
ws.loc[ws['pwsid'] == '010106001', 'cities_served'] = 'ledyard'

ws.loc[ws['pwsid'] == '010109005', 'counties_served'] = 'new london'
ws.loc[ws['pwsid'] == '010109005', 'cities_served'] = 'montville'

ws.loc[ws['pwsid'] == '010502003', 'counties_served'] = 'washington'
ws.loc[ws['pwsid'] == '010502003', 'cities_served'] = 'charlestown'
ws.loc[ws['pwsid'] == '010502002', 'counties_served'] = 'washington'
ws.loc[ws['pwsid'] == '010502002', 'cities_served'] = 'charlestown'

# we set the 'cities_served' and 'counties_served' to lower case:
ws.cities_served = ws.cities_served.str.lower() # set to lower case, as less annoying
ws.counties_served = ws.counties_served.str.lower() # set to lower case, as less annoying


In [94]:
# First Filter to select needed columns

select_features_ws = ['pwsid', 'pws_name', 'primacy_agency_code', 'pws_type_code', 
                      'gw_sw_code', # if the water system is considered having ground water (“gw”) or surface water (‘sw”) source under SDWA.
                     'owner_type_code', 'population_served_count', 'primary_source_code',
                      'is_wholesaler_ind', # whether the system is a wholesaler of water.
                     'is_school_or_daycare_ind', # if the water system’s primary service area is a school or daycare
                     'service_connections_count', 
                      'source_water_protection_code', # N: WS has not implemented source water protection according to state policy. Y: WS has substantially implemented.
                     # ! source_water_protection_code: most Y only after 01-JAN-2012
                      'cities_served', 'counties_served']

ws = ws.loc[:, select_features_ws] # keep only selected columns
ws.head()


,pwsid,pws_name,primacy_agency_code,pws_type_code,gw_sw_code,owner_type_code,population_served_count,primary_source_code,is_wholesaler_ind,is_school_or_daycare_ind,service_connections_count,source_water_protection_code,cities_served,counties_served
0,ME0004628,MACHIAS TRAILER PARK,ME,CWS,GW,P,65,GW,N,N,26,N,machias,washington
1,ME0092288,MARSH BROOK ESTATES,ME,CWS,GW,P,70,GW,N,N,28,N,sanford,york
2,ME0009198,TRAILS END STEAK HOUSE & TAVERN,ME,TNCWS,GW,P,390,GW,N,N,1,N,eustis,franklin
3,ME0094505,R & R VACATION HOME PARK,ME,TNCWS,GW,P,40,GW,N,N,1,N,naples,cumberland
4,ME0007311,NOKOMIS CAMPING AREA LLC,ME,TNCWS,GW,P,118,GW,N,N,1,N,harrison,cumberland


### Finding the Location of the Water Systems

TO BE DONE.

## Violations

We already filter the data by year, because we are only interested in recent violations. The number of observed violations greatly increased to reach a new plateau in 2009 (known from [previous work](https://github.com/de-la-viz/US-Public-Water-Systems/blob/master/US%20Drinking%20Water%20Quality%20Violations.ipynb)) because of the introduction of new guidelines and rules. We will thus focus on violations from 2009 onwards. 

In [96]:
# load the violations:
violations_raw = pd.read_csv('../data/violations_NewEngland.csv')
violations_raw.head().T

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (16,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4
Unnamed: 0,1,2,3,4,5
pwsid,ME0094672,ME0009683,ME0000625,ME0000625,ME0000625
violation_id,157508,60007,6318,6316,6315
facility_id,NaN,NaN,NaN,NaN,NaN
population_served_count,388,454,100,100,100
npm_candidate,N,N,N,N,N
pws_activity_code,A,A,A,A,A
pws_deactivation_date,NaN,NaN,NaN,NaN,NaN
primary_source_code,GW,GW,GW,GW,GW
pop_cat_5_code,1,1,1,1,1


In [121]:
# SOME FILTERING AND DATA CLEANING:

violations = violations_raw.copy()

# transform the dates to datetime:
violations.rtc_date = pd.to_datetime(violations.rtc_date)
violations.compl_per_begin_date = pd.to_datetime(violations.compl_per_begin_date)
violations.compl_per_end_date = pd.to_datetime(violations.compl_per_end_date)

violations.loc[:,'year'] = violations['compl_per_begin_date'].dt.year # year when the violation was discovered
violations.loc[:,'month'] = violations['compl_per_begin_date'].dt.month # month when the violation was discovered

# create new column with quarters:
def by_quarter(row):
    if row['month'] < 4:
        return 1
    elif row['month'] >= 4 and row['month'] < 7:
        return 2
    elif row['month'] >= 7 and row['month'] < 10:
        return 3
    else:
        return 4
violations.loc[:,'quarter'] = violations.apply(by_quarter, axis=1) 


select_features_viol = ['pwsid', 'violation_id', 'violation_code', 'violation_category_code', 
                       'is_health_based_ind', 'contaminant_code', 'is_major_viol_ind',
                       'rule_group_code',
                       'year', 'month', 'quarter']

violations = violations.loc[:, select_features_viol] # keep only selected columns

violations.head().T




,0,1,2,3,4
pwsid,ME0094672,ME0009683,ME0000625,ME0000625,ME0000625
violation_id,157508,60007,6318,6316,6315
violation_code,22,22,75,27,27
violation_category_code,MCL,MCL,Other,MR,MR
is_health_based_ind,Y,Y,N,N,N
contaminant_code,3100,3100,7500,2456,2950
is_major_viol_ind,NaN,NaN,NaN,Y,Y
rule_group_code,100,100,400,200,200
year,2008,2014,2014,2011,2011
month,6,8,10,1,1


TO DO: 

1. ADD THE CONTAMINANTS NAME
2. Filter for pesticide



## Merging all SDWIS Together

We first add the contaminants codes information to the violations.


In [ ]:
# merging contaminants codes with violations

violations = violations.merge(contaminant_codes, how='left', on='contaminant_code') # we want to keep all violations


We then merge the water systems and violations by _PWSID_ (it is not a 1 to 1 relation).

In [ ]:
# merging water systems with violations:

# 1 water system might see several violations, 
# and 1 violation might affect several water sytems (albeit it is rare)
NE_viol = water_system.merge(violations, how='outer')


In [1]:
# pesticides

In [ ]:
# the years 2013 to 2017 are estimates.
# the years 2015 to 2017 are not direclty accessible yet. 
# we first have to download them, then I reload them here:

pesticide_use_2015 = pd.read_csv('../data/pesticide_use/2015PreliminaryEstimates/EPest.county.estimates.2015.txt', sep='\t')
pesticide_use_2016 = pd.read_csv('../data/pesticide_use/2106PreliminaryEstimates/EPest.county.estimates.2016.txt', sep='\t')
pesticide_use_2017 = pd.read_csv('../data/pesticide_use/2017PreliminaryEstimatesNoCA/EPest.county.estimates_noCA.2017.txt', sep='\t')

# append to previous years:
pesticide_use_2009_17 = pesticide_use_2009_14.append(pesticide_use_2015, ignore_index=True)

print(pesticide_use_2009_17.shape)
pesticide_use_2009_17.head()